<a href="https://colab.research.google.com/github/kconstable/crypto-ensemble-model-predictions/blob/main/data_acquisition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries


In [ ]:
!pip install pytrends

In [3]:
import pandas as pd
import numpy as np
import requests
import time
import pickle
import re
from datetime import date, timedelta,datetime
from tabulate import tabulate
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from google.colab import files
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from google.colab import drive
drive.mount('/content/drive')

# from pygooglenews import GoogleNews
from pytrends.request import TrendReq
from bs4 import BeautifulSoup as bs
import requests as req


# alphavalue key
with open('/content/drive/MyDrive/Colab Notebooks/capstone2/data/av_key.txt') as f:
    key = f.read().strip()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Structured Features

### Fundamentals

In [ ]:
stock = 'VMW'
# current ratios
url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={stock}&apikey={key}'
r = requests.get(url)
data = r.json()

data

In [ ]:
url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={stock}&apikey={key}'
r = requests.get(url)
data = r.json()

# Earnings Per Share
print('date\t Reported EPS\t Estimated EPS\t Surprise %')
for earnings in data['quarterlyEarnings']:
  print(f"{earnings['reportedDate']}\t {earnings['reportedEPS']}\t {earnings['estimatedEPS']}\t\t {earnings['surprisePercentage']}")

  

In [ ]:
# income
stock= 'VMW'
url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={stock}&apikey={key}'
r = requests.get(url)
data = r.json()

print('date\t\t Net Income\t Gross Profit')
for income in data['quarterlyReports']:
  print(f"{income['fiscalDateEnding']}\t {income['comprehensiveIncomeNetOfTax']} \t{income['grossProfit']}")



In [ ]:
# cashflow
url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={stock}&apikey={key}'
r = requests.get(url)
data = r.json()

print('date\t\t ProfitLoss\t Cashflow')
for cf in data['quarterlyReports']:
  print(f"{cf['fiscalDateEnding']}\t {cf['profitLoss']}\t {cf['operatingCashflow']}")

### Economic

In [4]:
def get_economic_indicators(funct,key,interval=None,maturity=None,throttle=0):
  """
  Returns Economic Indicator Data with missing values interpolated between dates
  Monthly Data:
    NONFARM_PAYROLL, INFLATION_EXPECTATION,CONSUMER_SENTIMENT,UNEMPLOYMENT
  Quarterly Data: 
    GDP
  Daily Data:  
    FEDERAL_FUNDS_RATE = interval (daily,weekly,monthly)
    TREASURY_YIELD = interval (daily, weekly, monthly), 
                     maturity (3month, 5year, 10year, and 30year)
  """
  
  # query strings
  # Monthly Data:
  if funct in ['NONFARM_PAYROLL','INFLATION_EXPECTATION','CONSUMER_SENTIMENT','UNEMPLOYMENT']:
    url = f'https://www.alphavantage.co/query?function={funct}&apikey={key}'


  # Daily, Weekly,Monthly or Quarterly Data:
  # Interest Rates
  if funct in ['FEDERAL_FUNDS_RATE','REAL_GDP']:
    url = f'https://www.alphavantage.co/query?function={funct}&interval={interval}&apikey={key}'

  # Treasury Yield  
  if funct == 'TREASURY_YIELD':
    url = f'https://www.alphavantage.co/query?function={funct}&interval={interval}&maturity={maturity}&apikey={key}'

  # pull data
  r = requests.get(url)
  time.sleep(throttle)
  d = r.json()

  # convert to df
  df = pd.DataFrame(d['data'])

  # move date to a datetime index
  df.date = pd.to_datetime(df.date)
  df.set_index('date',inplace=True)

  # add the ticker name and frequency
  df['name'] = d['name']
  df['interval']=d['interval'] 

  # clean data & interpolate missing values
  # missing data encoded with '.'
  # change datatype to float
  df.replace('.',np.nan,inplace=True)
  df.value = df.value.astype('float')

  # missing data stats
  missing =sum(df.value.isna())
  total =df.shape[0]
  missing_pct = round(missing/total*100,2)

  # interpolate using the time index
  if missing >0:
    df.value.interpolate(method='time',inplace=True)
    action = 'interpolate-missing'
  else:
    action = 'none'

  # Print the results
  if maturity is not None:
    summary = ['Economic Indicator',funct+':'+maturity,str(total),str(missing),str(missing_pct)+'%',action]
  else:
    summary = ['Economic Indicator',funct,str(total),str(missing),str(missing_pct)+'%',action]

  return {'summary':summary,'data':df}

In [5]:
def plot_economic_indicators(df,yearfrom=None):
    """
    Plots candleskick plots: open/close/high/low prices + volume
    Input:
      df      : A dataframe with the price data
      yearfrom: filters the df by year
    """

    # get ticker name
    name = df.ticker.unique()[0]

    # filter by year if provided
    if yearfrom is not None:
      df = df[df.index.year >= yearfrom]
    
    # Create subplots
    fig = make_subplots(rows=7, cols=1, 
                        shared_xaxes=True, 
                        subplot_titles=('Closing Price','Volume','Non-Farm Payroll','Consumer Sentiment','Expected Inflation','Unemployment','GDP'),
                        vertical_spacing=0.07)

    # closing price
    fig.add_trace(go.Scatter(x=df.index,y=df.close,name = "Close Price",line_color = '#a8b8d0',fill='tozeroy'),
                row=1,col=1)
    
  
    # area chart for volume
    fig.add_trace(go.Scatter(x=df.index, y=df.volume, 
                fill='tozeroy',
                name='Volume',
                mode='none',
                fillcolor = 'orange'),
                row=2,col=1)
    
    # Non-Farm Payrolls
    fig.add_trace(go.Scatter(x=df.index, y=df.nfp,name='Non-Farm Payroll',line_color='orange'),
                  row=3,col=1)
    
    # Consumer Sentiment
    fig.add_trace(go.Scatter(x=df.index, y=df.cs,name='Consumer Sentiment',line_color='orange'),
                  row=4,col=1)

    # Expected Inflation    
    fig.add_trace(go.Scatter(x=df.index, y=df.infl,name='Expected Inflation',line_color='orange'),
                  row=5,col=1)
    
    # Unemployment    
    fig.add_trace(go.Scatter(x=df.index, y=df.unemployment,name='Unemployment',line_color = 'orange'),
                  row=6,col=1)
    
    # GDP    
    fig.add_trace(go.Scatter(x=df.index, y=df.gdp,name='GDP',line_color = 'orange'),
                  row=7,col=1)

    # Do not show OHLC's rangeslider plot 
    fig.update(layout_xaxis_rangeslider_visible=False)
    fig.update_layout(template='plotly_white',
                      title = f"Price, Volume and Economic Indicators: {name}",
                      width = 800,
                      height = 800)
          
    fig.show()

In [6]:
def plot_interest_rates(df,yearfrom):
  """
  """
  # filter by year if provided
  if yearfrom is not None:
      df = df[df.index.year >= yearfrom]


  # Interest Rates
  # add Fed Rate, then loop through the df columns to add any feature that is a yield
  # ********************************************************************************************
  fig = go.Figure()

  # fed funds rate
  fig.add_trace(
      go.Scatter(
          x=df.index, 
          y=df.ir,
          name='Fed Funds Rate',
          line_color='orange')
  )

  # add bond yields

  colors = ['brown','goldenrod','gold','yellow']
  rate_cols = [c for c in df.columns if 'yield' in c]
  for i,rate in enumerate(rate_cols):
    # Extract the name of the yield
    name = 'Yield:'+re.findall(r'\d+', rate)[0]+' Year'
    fig.add_trace(go.Scatter(x=df.index, y=df[rate],name=name,line_color = colors[i]))
  
  fig.update_layout(
      title = 'Interest Rates & Bond Yields',
      height = 500,
      width = 700,
      template = 'plotly_white'
  )
  
  fig.show()

### Commodities

In [7]:
def get_fx_rates(fx_from, fx_to,key,size='full',throttle=0):
  """
  """
  # query string
  url = f'https://www.alphavantage.co/query?function=FX_DAILY&from_symbol={fx_from}&to_symbol={fx_to}&apikey={key}&outputsize={size}'
  r = requests.get(url)

  if r.status_code == 200:
    data = r.json()
    df = pd.DataFrame(data['Time Series FX (Daily)']).T
    df.columns = ['open','high','low','close']
    df = df[['close']]
    
    # change data types
    df.index = pd.to_datetime(df.index)
    df.close = df.close.astype('float')

    # Calculate missing data
    missing = sum(df.close.isna())
    total = df.shape[0]
    missing_pct = round(missing/total*100,2)

    # rename the close column
    df.columns = [f"{fx_from}{fx_to}"]

    # Print the results
    summary = ['Ticker',f'{fx_from}/{fx_to}',str(total),str(missing),str(missing_pct)+'%','none']

  else:
    summary = ['FX Rates',f'{fx_from}/{fx_to}','Requst Error','all','100%','none']
    df = pd.DataFrame()
  
  return  {'summary':summary,'data':df}

In [8]:
def plot_tickers(df,tickers,title,yearfrom=None):
  """
  """

  # filter by year if provided
  if yearfrom is not None:
    df = df[df.index.year >= yearfrom]

  fig = go.Figure()

  colors = ['orange','gold','brown','goldenrod','yellow','crimson']
  for i,ticker in enumerate(tickers):
    fig.add_trace(
        go.Scatter(
            name =ticker,
            x = df.index,
            y = df[ticker],
            line_color = colors[i]
        )
      )

  fig.update_layout(
        title = title,
        width = 700,
        height = 500,
        template = 'plotly_white'
    )

  fig.show()

In [9]:
def get_ticker_data(symbol,key,outputsize='compact',throttle=0):
  """
  Returns daily data for a stock (symbol)
    outputsize: compact(last 100) or full (20 years)
    key: apikey
    symbols: OILK (oil ETF),BAR(gold ETF),VXZ (volatility ETF)
  """
  time.sleep(throttle)
  url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize={outputsize}&apikey={key}'
  r = requests.get(url)
  d = r.json()

  # extract data to a df
  df = pd.DataFrame(d['Time Series (Daily)']).T
  df.columns = ['open','high','low','close','volume']
  df['symbol'] = d['Meta Data']['2. Symbol']

  # change data types
  df.index = pd.to_datetime(df.index)

  # convert datatype to float
  for col in ['open','high','low','close','volume']:
    df[col] = df[col].astype('float')

  # Calculate missing data
  missing = sum(df.close.isna())
  total = df.shape[0]
  missing_pct = round(missing/total*100,2)

  # Print the results
  summary = ['Ticker',symbol,str(total),str(missing),str(missing_pct)+'%','none']

  return {'summary':summary,'data':df}

### Technicals

In [10]:
def calc_bollinger(df,feature,window=20,st=2):
  """
  Calculates bollinger bands for a price time-series.  Used for crypto currencies
  Input: 
    df     : A dataframe of time-series prices
    feature: The name of the feature in the df to calculate the bands for
    window : The size of the rolling window.  Defaults to 20 days with is standard
    st     : The number of standard deviations to use in the calculation. 2 is standard 
  Output: 
    Returns the df with the bollinger band columns added
  """
  # get the feature name
  if feature != 'close':
    name_u = f'b-upper-{feature}'
    name_l = f'b-lower-{feature}'
    name_m = f'b-middle-{feature}'
  else:
    name_u = 'b-upper'
    name_l = 'b-lower'
    name_m = 'b-middle'

  # rolling mean and stdev
  rolling_m  = df[feature].rolling(window).mean()
  rolling_st = df[feature].rolling(window).std()

  # add the upper/lower and middle bollinger bands
  df[name_u]  = rolling_m + (rolling_st * st)
  df[name_m] = rolling_m 
  df[name_l]  = rolling_m - (rolling_st * st)

In [11]:
def calc_rsi(df,feature='close',window=14):
  """
  Calculates the RSI for the input feature
  Input:
    df      : A dataframe with a time-series of prices
    feature : The name of the feature in the df to calculate the bands for
    window  : The size of the rolling window.  Defaults to 14 days which is standard
  Output: 
    Returns the df with the rsi band column added
  """
  # RSI
  # calc the diff in daily prices, exclude nan
  diff =df[feature].diff()
  diff.dropna(how='any',inplace=True)

  # separate positive and negitive changes
  pos_m, neg_m = diff.copy(),diff.copy()
  pos_m[pos_m<0]=0
  neg_m[neg_m>0]=0

  # positive/negative rolling means
  prm = pos_m.rolling(window).mean()
  nrm = neg_m.abs().rolling(window).mean()

  # calc the rsi and add to the df
  ratio = prm /nrm
  rsi = 100.0 - (100.0 / (1.0 + ratio))
  df['rsi']=rsi

In [12]:
def calc_macd(df,long=26,short=12,ma=9,feature='close'):
  """
  Calculates the MACD and signial for the input feature
  Input:
    df      : A dataframe with a time-series of prices
    feature : The name of the feature in the df to calculate the bands for
  Output: 
    Returns the df with the macd columns added
  """
  ema12 = df[feature].ewm(span=short,adjust=False).mean()
  ema26 = df[feature].ewm(span=long,adjust=False).mean()
  df['macd']=ema12-ema26
  df['macd_signal'] = df['macd'].ewm(span=ma,adjust=False).mean()

In [13]:
def calc_stoch_ossilator(df,window=14,feature = 'close'):
  """
  Calculates the stochastic ossilator signal for the input feature
  Params:
    df: dataframe of prices
    window: window lenght for high/low
    feature: open, high,low,close
  Reference:
    https://www.learnpythonwithrune.org/pandas-calculate-the-stochastic-oscillator-indicator-for-stocks/
  """

  # high/low values in the period
  df['stoch_high'] = df[feature].rolling(window).max()
  df['stoch_low']= df[feature].rolling(window).min()

  df['stoch_K'] = (df[feature] - df['stoch_low'])*100/(df['stoch_high'] - df['stoch_low'])
  df['stoch_D'] = df['stoch_K'].rolling(3).mean()

  return df

In [14]:
def plot_technicals(df,yearfrom=None):
  """
  Plots technical indicators
  Input:
    df: a dataframe with techical indicators
    yearfrom: filters the df by year
  """

  # get the symbol name
  name = df.ticker.unique()[0]

  # filter by year if provided
  if yearfrom is not None:
    df = df[df.index.year >= yearfrom]


  # make subplots
  fig = make_subplots(rows=2, cols=2,
                      shared_xaxes=True,
                      subplot_titles = ('Bollinger Bands','MACD','Stochastic Oscillator','RSI'),
                      vertical_spacing = 0.07)

  # boolinger bands
  # ********************************************************************************************
  fig.add_trace(go.Scatter(x=df.index,y=df['b-upper'],name='Bollinger-Upper',line_color="#DFEBF9"),row=1,col=1)
  fig.add_trace(go.Scatter(x=df.index,y=df['b-lower'],name='Bollinger-Lower',line_color='#DFEBF9',fill='tonexty'),row=1,col=1)
  fig.add_trace(go.Scatter(x=df.index,y=df.close,name='Closing Price',line_color='#a8b8d0'),row=1,col=1)


  # MACD
  # ********************************************************************************************
  fig.add_trace(go.Scatter(x=df.index,y=df['macd'],name='MACD',line_color='#F7DAC6'),row=1,col=2)
  fig.add_trace(go.Scatter(x=df.index,y=df['macd_signal'],name='MACD Signal',line_color='#E68A4C'),row=1,col=2)
  try:
    fig.add_trace(go.Bar(x=dff.index,y=df['macd_hist'],name='MACD Hist',marker_color='#E06D1F', marker_line_color='#E06D1F'),row=1,col=2)
  except:
    # skip mcad histogram for crypto
    print('')

  # Interest Rates
  # add Fed Rate, then loop through the df columns to add any feature that is a yield
  # ********************************************************************************************
  # fig.add_trace(go.Scatter(x=df.index, y=df.ir,name='Fed Funds Rate',line_color='#4B9D0C'),row=2,col=1)
  # colors = ['#0E340F','#29541F','#5E9C53','#A3E090','#537455','#3B463E']
  # rate_cols = [c for c in df.columns if 'yield' in c]
  # for i,rate in enumerate(rate_cols):
  #   # Extract the name of the yield
  #   name = 'Yield:'+re.findall(r'\d+', rate)[0]+' Year'
  #   fig.add_trace(go.Scatter(x=df.index, y=df[rate],name=name,line_color = colors[i]),row=2,col=1)

  # stochastic oscillator
  fig.add_trace(
      go.Scatter(
          name = 'stoch_K',
          x = df.index,
          y = df.stoch_K,
          marker_color = 'blue'
      ),row=2,col=1
  )
  fig.add_trace(
      go.Scatter(
          name = 'stoch_D',
          x = df.index,
          y = df.stoch_D,
          marker_color = 'skyblue'
      ),row=2,col=1
  )

  # # RSI
  # ********************************************************************************************
  fig.add_trace(go.Scatter(x=df.index,y=df['rsi'],name='RSI',line_color='#F00F3C'),row=2,col=2)
  fig.add_shape(type ='rect',
                x0=min(df.index),
                x1=max(df.index),
                y0=30.0,
                y1=70.0,
                line=dict(color='#F00F3C'),
                fillcolor='#F00F3C',
                opacity=0.1,
                row=2,col=2)
  fig.update_shapes(dict(xref='x',yref='y'),row=2,col=1)


  # Set template
  fig.update_layout(template = 'plotly_white',width= 1000,height=800,title ='Technical Indicators & Yields')
  fig.show()


In [15]:
def plot_bbands(df,feature):
  """
  """
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=df.index,y=df[f'b-upper-{feature}'],name='Bollinger-Upper',line_color="#DFEBF9"))
  fig.add_trace(go.Scatter(x=df.index,y=df[f'b-lower-{feature}'],name='Bollinger-Lower',line_color='#DFEBF9',fill='tonexty'))
  fig.add_trace(go.Scatter(x=df.index,y=df[feature],name=feature,line_color='#a8b8d0'))
  fig.update_layout(template = 'plotly_white',width =700,height=500,title = f'{feature}-Bollinger Bands')
  fig.show()

### Cypto

#### Crypto Prices

In [16]:
def get_crypto_spot(ticker,from_,to_,interval='1d'):
  """
  Get crypto spot rates from gate.io. Returns a dataframe of prices by date
  Params:
    ticker: crypto symbol
    from_: date from
    to_  : date to
    interval: 1d
  """
  # convert to unixtimestamp
  f = int(time.mktime(time.strptime(from_, "%Y-%m-%d")))
  t = int(time.mktime(time.strptime(to_, "%Y-%m-%d")))

  # request headers
  host = "https://api.gateio.ws"
  prefix = "/api/v4"
  headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}
  url = '/spot/candlesticks'
  query_param = f'currency_pair={ticker}_USDT&from={f}&to={t}&interval={interval}'

  # make request
  r = requests.request('GET', host + prefix + url + "?" + query_param, headers=headers)

  # if successful, convert to df,convert unixtimestamp, set the index & subset columns
  if r.status_code == 200:
    df = pd.DataFrame(r.json())
    df.columns = ['time','volume','close','high','low','open']
    df['date'] = df['time'].apply(lambda x:datetime.utcfromtimestamp(int(x)).strftime('%Y-%m-%d'))
    df['ticker'] = ticker
    df.set_index('date',inplace=True)
    df = df[['ticker','open','high','low','close','volume']]
    
    # change data types
    df.index = pd.to_datetime(df.index)

    # convert datatype to float
    for col in ['open','high','low','close','volume']:
      df[col] = df[col].astype('float')
  else:
    df = pd.DataFrame()
  
  return df

In [17]:
def get_crypto_spots(ticker,from_,to_,interval='1d'):
  """
  Make multiple requests to gate.io to get crypto prices (max 1,000 per request)
  This function breaks the date range into multiple requests and consolidates 
  if needed
  Params:
    ticker: crypto symbol
    from_ : date from as string
    to_   : date to as string
    interval: 1 day

  https://www.gate.io/docs/developers/apiv4/en/#get-futures-candlesticks
  """

  # calc the number of days in the requested range
  # max data per request is 1,000
  # multiple requests are needed if the delta is >1,000
  to_   = datetime.strptime(to_, '%Y-%m-%d')
  from_ = datetime.strptime(from_, '%Y-%m-%d')
  delta = to_-from_
  num_requests = delta.days/999

  # split into multiple requests if needed
  if num_requests >1:
    for i,req in enumerate(range(delta.days//999)):
      # first date range, create the first df
      if i ==0:
        # must adjust dates beween datetime objects
        # and strings for processing
        start = date(from_.year,from_.month,from_.day)
        end = start+timedelta(days=999)
        start_str = str(start)
        end_str = str(date(end.year,end.month,end.day))

        # request data
        df = get_crypto_spot(ticker,start_str,end_str,interval)

      else: #append to existing df
        start = end + timedelta(days=1)
        end = start + timedelta(days=999)
        start_str = str(date(start.year,start.month,start.day))
        end_str = str(date(end.year,end.month,end.day))

        # make request, append results to df
        df_tmp = get_crypto_spot(ticker,start_str,end_str,interval)
        df = pd.concat([df,df_tmp])

    # request the data for the remaining range
    start = end + timedelta(days=1)
    start_str = str(date(start.year,start.month,start.day))
    end_str = str(date(to_.year,to_.month,to_.day))

    # make request, append data to df
    df_tmp = get_crypto_spot(ticker,start_str,end_str,interval)
    df = pd.concat([df,df_tmp])

  else:
    # if only one request is needed (less than 1000 days in range)
    start = str(date(from_.year,from_.month,from_.day))
    end = str(date(to_.year,to_.month,to_.day))
    df = get_crypto_spot(ticker,start,end,interval)


  # Calculate missing data
  missing = sum(df['close'].isna())
  total = df.shape[0]
  missing_pct = round(missing/total*100,2)

  # Print the results
  summary = ['Ticker',f'{ticker}-Spots',str(total),str(missing),str(missing_pct)+'%','none']

  return {'summary':summary,'data':df}


#### Crypto Futures
+ https://www.cnbc.com/quotes/@BTC.1

In [18]:
def get_crypto_futures(ticker,from_,to_,interval='1d'):
  """
  Get crypto futures prices from gate.io. Returns a dataframe with open,high,low,close, and volume
  Params:
    ticker: crypto symbol
    from_: from date yyyy-mm-dd
    to_  : to date yyyy-mm-dd
    interval: 1d
  https://www.gate.io/docs/developers/apiv4/en/#get-futures-candlesticks
  """

  # request headers
  host = "https://api.gateio.ws"
  prefix = "/api/v4"
  headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}
  url = '/futures/usdt/candlesticks'

  # min date of futures is Nov 18,2019
  from_ = max(from_,'2019-11-18')

  # convert to unixtimestamp
  f = time.mktime(time.strptime(from_, "%Y-%m-%d"))
  t = time.mktime(time.strptime(to_, "%Y-%m-%d"))
  query_param = f'contract={ticker}_USDT&interval={interval}&from={f}&to={t}'


  # request the data
  r = requests.request('GET', host + prefix + url + "?" + query_param, headers=headers)

  # if successful, convert to df,convert unixtimestamp, set the index & subset columns
  if r.status_code == 200:
    df = pd.DataFrame(r.json())
    df['date'] = df['t'].apply(lambda x:datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))
    df.set_index('date',inplace=True)
    df = df[['o','h','l','c','v']]
    df.columns = ['futures_open','futures_high','futures_low','futures_close','futures_volume']

    # change data types
    df.index = pd.to_datetime(df.index)

    # convert datatype to float
    for col in ['futures_open','futures_high','futures_low','futures_close','futures_volume']:
      df[col] = df[col].astype('float')

    # Calculate missing data
    missing = sum(df['futures_close'].isna())
    total = df.shape[0]
    missing_pct = round(missing/total*100,2)

    # Print the results
    summary = ['Ticker',f'{ticker}-Futures',str(total),str(missing),str(missing_pct)+'%','none']

  else:
    summary = ['Ticker',f'{ticker}-Futures',f'Error:{r.status_code}','all','100%','none']
    df = pd.DataFrame()
  
  return {'summary':summary,'data':df}

#### Fear and Greed Index for Bitcoin

In [19]:
def get_fear_greed_index(index_date = None):
  """
  """
  if index_date is None:
    # get all data available
    url ="https://api.alternative.me/fng/?limit=0&date_format=cn"
  else:
    # get the latest three data-points
    url = "https://api.alternative.me/fng/?limit=3&date_format=cn"
  
  # make request
  r = requests.get(url)

  # check the status of the request 
  if r.status_code ==200:

    # convert data to a dataframe
    data = r.json()
    df = pd.DataFrame(data['data'])
    df['date'] = pd.to_datetime(df['timestamp'])
    df.set_index('date',inplace=True)
    df = df[['value','value_classification']]
    df.columns = ['idx_fear_greed','idx_classification']

    # convert datatype to float
    df['idx_fear_greed'] = df['idx_fear_greed'].astype('float')

    # Calculate missing data
    missing = sum(df['idx_fear_greed'].isna())
    total = df.shape[0]
    missing_pct = round(missing/total*100,2)

    if index_date is not None:
      # filter for the index date
      df = df[index_date]

    # Print the results
    summary = ['Ticker','Fear-Greed-Index',str(total),str(missing),str(missing_pct)+'%','none']

  else: #request error
      summary = ['Ticker','Fear-Greed-Index','Request Error','all','100%','none']
      df = pd.DataFrame()

  return {'summary':summary,'data':df}

#### Google Trends

In [20]:
def get_google_trends(search_terms_list,timeframe ='today 5-y' ):
  """
  Get search term frequency from google
  Params:
    search_terms: list of google search terms (max 5)
    timeframe: 'today 5-y', '2016-12-14 2017-01-25'
  Reference: https://pypi.org/project/pytrends/
  """
  # connect to google trends
  pytrends = TrendReq(hl='en-US', tz=360)

  # search for the terms
  pytrends.build_payload(search_terms_list,timeframe=timeframe)

  # get data over time
  df = pytrends.interest_over_time()

  df.columns = ['google_trends','isPartial']
  df = df[['google_trends']]

  # Calculate missing data
  missing = sum(df['google_trends'].isna())
  total = df.shape[0]
  missing_pct = round(missing/total*100,2)

  # Print the results
  summary = ['Ticker',f'Google Trends',str(total),str(missing),str(missing_pct)+'%','none']

  return {'summary':summary,'data':df}

In [21]:
def plot_google_trends(df,from_=None,to_=None):
  """
  """
  if from_ is not None and to_ is not None:
    df = df.loc[from_:to_,:].copy()

  fig = go.Figure()

  fig.add_trace(
      go.Scatter(
          name = 'Search Volume',
          x = df.index,
          y = df.google_trends,
          line_color = 'orange',
          fill = 'tozeroy'
      )
  )
  fig.update_layout(
      title = 'Google Trends: Search Term Volumes',
      template = 'plotly_white',
      width = 700,
      height = 500,
      yaxis_title = 'Number of searches per day'
  )
  fig.show()

In [22]:
def plot_fear_greed_index(df,from_=None,to_=None):
  """
  """

  if from_ is not None and to_ is not None:
    df = df.loc[from_:to_,:].copy()
  
  fig = go.Figure()

  fig.add_trace(
      go.Scatter(
          name = 'Bitcoin Fear-Greed Index',
          x = df.index,
          y = df.idx_fear_greed,
          line_color = 'orange'
      )
  )
  fig.add_trace(
      go.Scatter(
          name = 'upper-bollinger',
          x = df.index,
          y = df['b-upper-idx_fear_greed'],
          line_color = '#DFEBF9'
      )
  )

  fig.add_trace(
      go.Scatter(
          name = 'lower-bollinger',
          x = df.index,
          y = df['b-lower-idx_fear_greed'],
          line_color = '#DFEBF9',
          fill = 'tonexty'
      )
  )
  fig.update_layout(
      title = 'Bitcoin Fear-Greed Index',
      template = 'plotly_white',
      width = 700,
      height = 500
  )
  fig.show()


# Get Consolidated Crypto Data

#### Data Config

In [23]:
ticker = 'BTC'
search_terms = ['how to buy bitcoin']
path = f'/content/drive/MyDrive/Colab Notebooks/capstone2/data/'

config_crypto = {
    'process':True,
    'data':{'spot':ticker,'futures':ticker,'index':'fear_greed_index','peers':['ETH','DOGE','LTC']}
}
config_commodities = {
    'process':True,
    'data':['GLD','OIL','BOIL','VXX','SPY','XLE','QQQ']
    }
config_technicals = {
    'process':True,
    'data':[{'name':'bbands','window':20,'stdev':2},{'name':'rsi','window':14},{'name':'stoch','window':14},{'name':'macd','long':26,'short':12,'ma':9}]
}     
config_sentiment = {
    'process':True,
    'data':[{'name':'news','path':f'{path}{ticker}_sentiment.pickle'},{'name':'news_volume','path':f'{path}{ticker}_news_counts.pickle'},{'name':'google_trends','search_terms':search_terms}]
}
config_fx = {
    'process':True,
    'data':[{'from':'USD','to':'EUR'},{'from':'USD','to':'JPY'},{'from':'USD','to':'GBP'}]
}
config_econ = {
    'process':True,
    'data':{'TREASURY_YIELD':[{'interval':'daily','maturity':'5year','name':'yield5y'},
                            {'interval':'daily','maturity':'10year','name':'yield10y'},
                            {'interval':'daily','maturity':'30year','name':'yield30y'},
                            {'interval':'daily','maturity':'3month','name':'yield3m'}
                            ],
          'FEDERAL_FUNDS_RATE':{'interval':'daily','name':'ir'},
          'NONFARM_PAYROLL':{'interval':'monthly','name':'nfp'},
          'REAL_GDP':{'interval':'quarterly','name':'gdp'},
          'UNEMPLOYMENT':{'interval':'monthly','name':'unemployment'},
          'CONSUMER_SENTIMENT':{'interval':'monthly','name':'cs'},
          'INFLATION_EXPECTATION':{'interval':'monthly','name':'infl'},  
  }
}

# combine
config = {'Crypto':config_crypto,
          'Commodities':config_commodities,
          'Technical':config_technicals,
          'Sentiment':config_sentiment,
          'FX':config_fx,
          'Economic':config_econ
          }

#### Data Consolidation

In [24]:
def get_consolidated_data(config,key,from_,to_,interval='1d',throttle=30,dropna=False):
  """
  """

  # Result header and accumulator
  header = ['Type','Data','Total','Missing',' % ','Action']
  summary =[]


  # get spot crypto data, add month/weekday cols
  # ****************************************************************************
  # get spot crypto prices
  if config['Crypto']['process']:
    try:
      results = get_crypto_spots(config['Crypto']['data']['spot'],from_,to_,interval)
      dff = results['data']
      dff['month'] =dff.index.month
      dff['weekday']=dff.index.weekday
      summary.append(results['summary'])
      print(f"Complete:===>Spot Crypto:{config['Crypto']['data']['spot']}")
    except:
      print(f"Error:=====>Spot Crypto:{config['Crypto']['data']['spot']}")
      return ''

  # get futures prices
  # ****************************************************************************
  if config['Crypto']['process']:
    try:
      # get data between from and to
      # earliest date available is nov 18, 2019
      results = get_crypto_futures(config['Crypto']['data']['futures'],from_,to_,interval)

      # append results
      summary.append(results['summary'])
      dff = dff.join(results['data'],how='left')

      print(f"Complete:===>Futures Crypto:{config['Crypto']['data']['futures']}")
    except:
      print(f"Error:=====>Futures Crypto:{config['Crypto']['data']['futures']}")
    
    # get bitcoin index
    # **************************************************************************
  if config['Crypto']['process']:
    try:
      # get fear and greed index
      results = get_fear_greed_index()

      #append the results
      summary.append(results['summary'])
      dff = dff.join(results['data'],how='left')
      print('Complete:===>Fear-Greed Index')
    except:
      print('Error:=====>Fear-Greed Index')
    
  # get bitcoin peers
  # **************************************************************************
  if config['Crypto']['process']:
    for ticker in config['Crypto']['data']['peers']:
      try:
        # get spot prices for each peer
        results = get_crypto_spots(ticker,from_,to_,interval)

        # rename and subset the columns
        data = results['data']
        data.rename(columns = {'close':ticker},inplace=True)
        data = pd.DataFrame(data[ticker])

        #append the results
        summary.append(results['summary'])
        dff = dff.join(data,how='left')
        print(f'Complete:===>Peer:{ticker}')
      except:
        print(f'Error:=====>Peer:{ticker}')    

  # Sentiment
  # ****************************************************************************
  if config['Sentiment']['process']:
    for sent in config['Sentiment']['data']:
      if sent['name']=='google_trends' and len(sent['search_terms'])>=1:
        try:
          results = get_google_trends(sent['search_terms'])

          # update the action log
          if results['summary'][5]=='none':
            # there were no missing values to interpolate
            results['summary'][5]=f"fill weekly to daily"
          else:
            # missing values were interpolated
            results['summary'][5]=f"{results['summary'][5]} + fill weekly to daily"
      
          # fill in missing days (measured weekly)
          df = results['data']
          days = pd.date_range(start = min(df.index),end =max(dff.index),freq='D')
          df =df.reindex(days,method = 'bfill')

          # join with main df
          dff = dff.join(df,how='left')
          summary.append(results['summary'])

          # calc bbands for google trend data
          calc_bollinger(dff,'google_trends')
          print('Complete:===>Google Trends Data')

        except:
          print('Error:=====>Google Trends Data')

      if sent['name']=='news' and sent['path'] is not None:
        df_news = pd.read_pickle(sent['path'])
        dff =dff.join(df_news,how='left')

        # Calc bbands for sentiment moving average line
        calc_bollinger(dff,'ma_sentiment_10')
        print("Complete:===>News Sentiment")

      if sent['name']=='news_volume' and sent['path'] is not None:
        df_news = pd.read_pickle(sent['path'])
        dff =dff.join(df_news,how='left')

        # calc bands for news volume and title sentiment
        calc_bollinger(dff,'ma_news_count')
        calc_bollinger(dff,'ma_sentiment_title')
        print('Complete:===>News Count')

  # FX rates
  # ****************************************************************************
  if config['FX']['process']:
    for fx in config['FX']['data']:
      try:
        results = get_fx_rates(fx['from'],fx['to'],key)
        df = results['data']
        summary.append(results['summary'])
        dff = dff.join(df,how='left')
        print(f"Complete:===>FX:{fx['from']}->{fx['to']}")
      except:
        summary.append(results['summary'])
        print(f"Error:=====>FX:{fx['from']}->{fx['to']}")

  # Get Commodity prices
  # ****************************************************************************
  if config['Commodities']['process']:
    for commodity in config['Commodities']['data']:
      try:
        # get prices
        results = get_ticker_data(commodity,key,'full',throttle)
        df = results['data']
        summary.append(results['summary'])
        print(f'Complete:===>Commodity:{commodity}')

        # rename close to commodity name, remove unneeded columns and join with 
        # the crypto prices by date
        df.rename(columns={'close':commodity},inplace=True)
        df.drop(['open','high','low','volume','symbol'],axis=1,inplace=True)
        dff = dff.join(df,how='left')
      except:
        print(f"Error:=====>Commodity:{commodity}")
    
  # Technical Indicators
  # ****************************************************************************
  if config['Technical']['process']:
    for tech in config['Technical']['data']:
      if tech['name']=='bbands':
        calc_bollinger(dff,'close',tech['window'],tech['stdev'])
        print('Complete:===>Bollinger Bands Calculated')
      elif tech['name']=='rsi':
        calc_rsi(dff,'close',tech['window'])
        print('Complete:===>RSI Calculated')
      elif tech['name']=='macd':
        calc_macd(dff,tech['long'],tech['short'],tech['ma'])
        print('Complete:===>MACD Calculated')
      elif tech['name']=='stoch':
        calc_stoch_ossilator(dff,tech['window'])
        print('Complete:===>Stochastic Oscillator Calculated')
      else:
        print('')

  # Economic Indicators
  # ****************************************************************************
  # loop through the config to pull the requested data
  if config['Economic']['process']:
    for indicator,values in config['Economic']['data'].items():
      if indicator == 'TREASURY_YIELD':
        for tr in values:
          try:
            # treasury yields have a maturity component
            results = get_economic_indicators(indicator,key,interval=tr['interval'],maturity=tr['maturity'],throttle=throttle)
            summary.append(results['summary'])

            # extract the data, rename columns
            df = results['data']
            df.rename(columns={"value": tr['name']},inplace=True)
            df.drop(['name', 'interval'], axis=1,inplace = True)

            # append to consolidated dff
            dff = dff.join(df,how='left')
            print(f"Complete:===>{indicator}:{tr['maturity']}")

          except:
            print(f"Error:=====>{indicator}:{tr['maturity']}")
    
      else: 
        # daily
        if values['interval']=='daily':
          try:
            results = get_economic_indicators(indicator,key,interval=values['interval'],throttle=throttle)
            df = results['data']
            summary.append(results['summary'])
            
            df.rename(columns={"value": values['name']},inplace=True)
            df.drop(['name', 'interval'], axis=1,inplace = True)
            dff = dff.join(df,how='left')
            print(f"Complete:===>{indicator}")
          except:
            print(f"Error:=====>{indicator}")
    
        else: 
          try:
            # weekly, monthly or quarterly
            results =get_economic_indicators(indicator,key,interval=values['interval'],throttle=throttle)
            df = results['data']
            
            # reindex to daily, fill missing values forward (index is in reverse order)
            days = pd.date_range(start = min(df.index),end =max(dff.index),freq='D')
            df =df.reindex(days,method = 'bfill')

            # update the action log
            if results['summary'][5]=='none':
              # there were no missing values to interpolate
              results['summary'][5]=f"fill {values['interval']} to daily"
            else:
              # missing values were interpolated
              results['summary'][5]=f"{results['summary'][5]} + fill {values['interval']} to daily"

            summary.append(results['summary'])
        
            # join with the other data
            df.rename(columns={"value": values['name']},inplace=True)
            df.drop(['name', 'interval'], axis=1,inplace = True)
            dff = dff.join(df,how='left')
            print(f"Complete:===>{indicator}")
          except:
            print(f"Error:=====>{indicator}")


  # Fill in any missing data after joining all datasets
  dff.fillna(method='ffill',inplace=True,axis = 0)

  # drop rows with missing commodity prices
  if dropna:
    dff.dropna(how='any',inplace=True)

  # print the results table
  print("\n\n")
  print(tabulate(summary,header))

  return dff
  

In [25]:

# get the consolidated data
df =get_consolidated_data(config,key,'2016-01-01','2021-10-25')

# save to google drive
df.to_pickle(f'{path}{ticker}_market_data.pickle')



Complete:===>Spot Crypto:BTC
Complete:===>Futures Crypto:BTC
Complete:===>Fear-Greed Index
Complete:===>Peer:ETH
Complete:===>Peer:DOGE
Complete:===>Peer:LTC
Complete:===>News Sentiment
Complete:===>News Count
Complete:===>Google Trends Data
Complete:===>FX:USD->EUR
Complete:===>FX:USD->JPY
Complete:===>FX:USD->GBP
Complete:===>Commodity:GLD
Complete:===>Commodity:OIL
Complete:===>Commodity:BOIL
Complete:===>Commodity:VXX
Complete:===>Commodity:SPY
Complete:===>Commodity:XLE
Complete:===>Commodity:QQQ
Complete:===>Bollinger Bands Calculated
Complete:===>RSI Calculated
Complete:===>Stochastic Oscillator Calculated
Complete:===>MACD Calculated
Complete:===>TREASURY_YIELD:5year
Complete:===>TREASURY_YIELD:10year
Complete:===>TREASURY_YIELD:30year
Complete:===>TREASURY_YIELD:3month
Complete:===>FEDERAL_FUNDS_RATE
Complete:===>NONFARM_PAYROLL
Complete:===>REAL_GDP
Complete:===>UNEMPLOYMENT
Complete:===>CONSUMER_SENTIMENT
Complete:===>INFLATION_EXPECTATION



Type                Data        

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2125 entries, 2016-01-01 to 2021-10-25
Data columns (total 69 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ticker                       2125 non-null   object 
 1   open                         2125 non-null   float64
 2   high                         2125 non-null   float64
 3   low                          2125 non-null   float64
 4   close                        2125 non-null   float64
 5   volume                       2125 non-null   float64
 6   month                        2125 non-null   int64  
 7   weekday                      2125 non-null   int64  
 8   futures_open                 708 non-null    float64
 9   futures_high                 708 non-null    float64
 10  futures_low                  708 non-null    float64
 11  futures_close                708 non-null    float64
 12  futures_volume               708 non-null    float64
 13  

# Exploratory Data Analysis

#### Technical Indicators

In [ ]:
plot_technicals(df,2021)

#### Economic Indicators

In [ ]:
plot_economic_indicators(df,2019)

In [ ]:
plot_interest_rates(df,2016)

In [ ]:

plot_tickers(df,['USDEUR','USDGBP'],'FX Rates')
plot_tickers(df,['GLD','OIL','BOIL','VXX','QQQ','SPY'],'Commodity & Index Prices')

#### Crypto Fear and Greed Index

In [ ]:
plot_fear_greed_index(df,'2021-01-01','2021-10-24')

#### Google Trends Search History

In [ ]:
plot_google_trends(df,'2016-10-01','2021-10-24')